In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
df=pd.read_csv('../input/feedback-prize-2021/train.csv')
df.head()

In [ ]:
df.shape

In [ ]:

px.histogram(df,x='discourse_type')

In [ ]:
df['length_of_text_char']=list(map(len,df['discourse_text']))
df['length_of_text_word']=list(map(lambda x:len(x.split(' ')),df['predictionstring']))


In [ ]:
px.histogram(df,x='length_of_text_char',marginal='box')

In [ ]:
px.histogram(df,x='length_of_text_word',marginal='box')

In [ ]:
df['char/word']=df['length_of_text_char']/df['length_of_text_word']
px.histogram(df,x='char/word',marginal='box')

# Observe any changes in statistics on each target , length of the word and sentence can "have" a influence on the rating

# Lead

In [ ]:
crop=df[df['discourse_type']=='Lead']
crop2=df[df['discourse_type']!='Lead']
average=np.mean(crop2['length_of_text_word'])-np.mean(crop['length_of_text_word'])
if average<0:new='increased'
else:new='decreased'
print('Shift of length of text',np.abs(average),'words',new)
px.histogram(crop,x='length_of_text_word',marginal='box')

# Position

In [ ]:
crop=df[df['discourse_type']=='Position']
average=np.mean(crop2['length_of_text_word'])-np.mean(crop['length_of_text_word'])
if average<0:new='increased'
else:new='decreased'
print('Shift of length of text',np.abs(average),'words',new)
px.histogram(crop,x='length_of_text_word',marginal='box')

# Evidence

In [ ]:
crop=df[df['discourse_type']=='Evidence']
crop2=df[df['discourse_type']!='Evidence']
average=np.mean(crop2['length_of_text_word'])-np.mean(crop['length_of_text_word'])
if average<0:new='increased'
else:new='decreased'
print('Shift of length of text',np.abs(average),'words',new)
px.histogram(crop,x='length_of_text_word',marginal='box')


# Claim

In [ ]:
crop=df[df['discourse_type']=='Claim']
crop2=df[df['discourse_type']!='Claim']
average=np.mean(crop2['length_of_text_word'])-np.mean(crop['length_of_text_word'])
if average<0:new='increased'
else:new='decreased'
print('Shift of length of text',np.abs(average),'words',new)
px.histogram(crop,x='length_of_text_word',marginal='box')

# Concluding Statement

In [ ]:
crop=df[df['discourse_type']=='Concluding Statement']
crop2=df[df['discourse_type']!='Concluding Statement']
average=np.mean(crop2['length_of_text_word'])-np.mean(crop['length_of_text_word'])
if average<0:new='increased'
else:new='decreased'
print('Shift of length of text',np.abs(average),'words',new)
px.histogram(crop,x='length_of_text_word',marginal='box')

# Counterclaim

In [ ]:
crop=df[df['discourse_type']=='Counterclaim']
crop2=df[df['discourse_type']!='Counterclaim']
average=np.mean(crop2['length_of_text_word'])-np.mean(crop['length_of_text_word'])
if average<0:new='increased'
else:new='decreased'
print('Shift of length of text',np.abs(average),'words',new)
px.histogram(crop,x='length_of_text_word',marginal='box')

# Rebuttal

In [ ]:
crop=df[df['discourse_type']=='Rebuttal']
crop2=df[df['discourse_type']!='Rebuttal']
average=np.mean(crop2['length_of_text_word'])-np.mean(crop['length_of_text_word'])
if average<0:new='increased'
else:new='decreased'
print('Shift of length of text',np.abs(average),'words',new)
px.histogram(crop,x='length_of_text_word',marginal='box')

# Conclusions from this part
Differnet classes show great swingsin  max number of words , the shifts were average words

quick plot over this by observing a bar plot of the number of words on each class

In [ ]:
classes=set(list(df['discourse_type']))
hist=dict()
for c in classes:
    crop=df[df['discourse_type']==c]
    ma=np.max(crop['length_of_text_word'])
    hist[c]=ma
hist

In [ ]:
T={'Class':list(hist.keys()),'Count':list(hist.values())}
px.bar(T, x='Class', y='Count')

# Tokenize the words

In [ ]:
sentences=list(df['discourse_text'])
from tqdm import tqdm
from nltk.tokenize import word_tokenize
print('Tokenizing')
tokenized=list(map(word_tokenize,tqdm(sentences)))


# See any  imbalance in the frequency of the words

In [ ]:
histogram=dict()
print('Taking out the freuqency of each word')
for k in tqdm(tokenized):
    for word in k:
        try:
            histogram[word]+=1
        except:
            histogram[word]=1
        

In [ ]:
print('We have',len(histogram.keys()),'unique words and symbols in our dataset')

In [ ]:
print('Percentage of imbalance in the data',np.var(list(histogram.values()))/(np.max(list(histogram.values())-np.min(list(histogram.values()))))**2*100,'Percentage')

# StopWords
Certain classes may have a high stop word count, 

what are stopwords ?

They are words like 'a','an','the',etc and have no meaning in themselves a high stopword count may have a infleuence on the class

spacy library already has many stopwords installed 

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS as stop_words
print('We have ,',len(stop_words),'stopwords')

In [ ]:
#calculates percentage of stopwords in each essay
def calculate(words):
    n=0
    for word in words:
        if word in stop_words:
            n+=1
    return n/len(words)*100
df['stopwords']=list(map(calculate,tokenized))


In [ ]:
h=df.groupby('discourse_type').mean()

In [ ]:
h

# Plot the percentage of stopwords

50 percent of the essays are filled with just stopwords , they dont seem to change much with the class though

In [ ]:
print('Average percentage of stopwords,',np.mean(h['stopwords']))
px.bar(h,x='stopwords')

# Conclusions from this part
We might consider taking samples with too many stopwords as outliers

In [ ]:
px.histogram(df,x='stopwords',marginal='box')

# Removing Outlier

In [ ]:
percentile25 = df['stopwords'].quantile(0.25)
percentile75 = df['stopwords'].quantile(0.75)

In [ ]:
iqr=(percentile75-percentile25)

In [ ]:
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr
print(upper_limit,lower_limit)

In [ ]:
df['outlier1']=(df['stopwords']>upper_limit).astype(int)
df['outlier2']=(df['stopwords']<lower_limit).astype(int)
df['outlier']=np.minimum(df['outlier1']+df['outlier2'],1)
df['outlier']=np.where(df['outlier']==1,True,False)
px.histogram(df,x='outlier')

# Outlier removal based on sentence length

In [ ]:
percentile25 = df['length_of_text_word'].quantile(0.25)
percentile75 = df['length_of_text_word'].quantile(0.75)
iqr=(percentile75-percentile25)
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr
print(upper_limit,lower_limit)

df['outlier1']=(df['length_of_text_word']>upper_limit).astype(int)
df['outlier2']=(df['length_of_text_word']<lower_limit).astype(int)
df['outlier3']=np.minimum(df['outlier1']+df['outlier2'],1)
df['outlier3']=np.where(df['outlier']==1,True,False)
px.histogram(df,x='outlier3')

# Merge Everything

In [ ]:
df['outliers']=np.minimum(df['outlier'].astype(int)+df['outlier3'].astype(int),1).astype(bool)
px.histogram(df,x='outliers')

# Thank YOU

Work is still in progress and i will be updating this kernel , please give me feedbacks and improvements you want ,so that i can make this kernel better